In [ ]:
%run edit_video.py \
    --dit-weight "/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt" \
    --video-size 512 512 \
    --video-length 25 \
    --infer-steps 28 \
    --inject 0 \
    --seed 42 \
    --embedded-cfg-scale 3.5 \
    --flow-shift 7.0 \
    --flow-reverse \
    --use-cpu-offload \
    --use-fp8 \
    --save-path ./results/V2VBench/rf-solver_2order \
    --dataset "V2VBench" \
    --gamma 0.5 \
    --start-timestep 0\
    --stop-timestep 0.25 \
    --eta-reverse 0.9

In [1]:
%run edit_video.py \
    --dit-weight "/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt" \
    --video-size 512 512 \
    --video-length 25 \
    --infer-steps 28 \
    --prompt "a grey car navigates a curvy road surrounded by green grass trees and mountains." \
    --target-prompt "a red sports-car navigates a curvy road surrounded by green grass trees and mountains." \
    --inject 0 \
    --seed 42 \
    --embedded-cfg-scale 3.5 \
    --flow-shift 7.0 \
    --flow-reverse \
    --use-cpu-offload \
    --use-fp8 \
    --save-path ./results/rf_inversion \
    --inverse-video-path "/data/chx/V2VBench/videos/car-turn.mp4" \
    --gamma 0.5 \
    --start-timestep 0\
    --stop-timestep 0.25 \
    --eta-reverse 0.9

/data/chx/anaconda3/envs/HunyuanVideo_rf_inv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-04 21:42:20.359 | INFO     | hyvideo.inference:from_pretrained:155 - Got text-to-video model root path: ckpts
2025-04-04 21:42:20.361 | INFO     | hyvideo.inference:from_pretrained:190 - Building model...


detect you are not use the latest yunchang. Please install yunchang>=0.4.0
{'apply_final_norm': False,
 'batch_size': 1,
 'cfg_scale': 1.0,
 'dataset': None,
 'decay_eta': False,
 'denoise_type': 'flow',
 'disable_autocast': False,
 'dit_weight': '/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt',
 'embedded_cfg_scale': 3.5,
 'eta_decay_power': 1.0,
 'eta_reverse': 0.9,
 'feature_path': 'feature',
 'flow_reverse': True,
 'flow_shift': 7.0,
 'flow_solver': 'euler',
 'gamma': 0.5,
 'hidden_state_skip_layer': 2,
 'infer_steps': 28,
 'inject': 0,
 'inverse_video_path': '/data/chx/V2VBench/videos/car-turn.mp4',
 'latent_channels': 16,
 'linear_schedule_end': 25,
 'load_key': 'module',
 'model': 'HYVideo-T/2-cfgdistill',
 'model_base': 'ckpts',
 'model_resolution': '540p',
 'name_suffix': '',
 'neg_prompt': None,
 'num_videos': 1,
 'precision': 'bf16',
 'prompt': 'a grey car navigates a curvy road surrounded by green grass trees '
        

/home/chx/mySrc/HunyuanVideo_rf_inv/hyvideo/modules/fp8_optimization.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fp8_map = torch.load(fp8_map_path, map_location=lam

loading vae


/home/chx/mySrc/HunyuanVideo_rf_inv/hyvideo/vae/__init__.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(vae_ckpt, map_location=vae.device)
2025-04-04

...

 89%|████████▉ | 25/28 [00:40<00:04,  1.62s/it]

Step 25: t_curr = 456.5217590332031, t_prev = 350.0


 93%|█████████▎| 26/28 [00:42<00:03,  1.61s/it]

Step 26: t_curr = 350.0, t_prev = 205.88235473632812


 96%|█████████▋| 27/28 [00:44<00:01,  1.64s/it]
2025-04-04 21:47:33.152 | INFO     | hyvideo.inference:predict:721 - Success, time: 123.67549896240234
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-04-04 21:47:35.685 | INFO     | __main__:process_sample:181 - 保存单次结果到: ./results/rf_inversion/car-turn_single_video_20250404-214733_s42.mp4
2025-04-04 21:47:35.697 | INFO     | __main__:main:206 - 单视频处理完成


In [1]:
%run edit_video.py \
    --dit-weight "/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt" \
    --video-size 512 512 \
    --video-length 53 \
    --infer-steps 28 \
    --prompt "A black swan swimming in a pond with lush greenery in the background." \
    --target-prompt "A majestic flamingo swimming in a pond with lush greenery in the background." \
    --inject 0 \
    --seed 42 \
    --embedded-cfg-scale 3.5 \
    --flow-shift 7.0 \
    --flow-reverse \
    --use-cpu-offload \
    --use-fp8 \
    --save-path ./results/rf_inversion \
    --inverse-video-path "/data/chx/BalanceCC-rename/Result/Animal/blackswan.mp4" \
    --gamma 0.5 \
    --start-timestep 0\
    --stop-timestep 0.25 \
    --eta-reverse 0.9

/data/chx/anaconda3/envs/HunyuanVideo_rf_inv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-03 09:37:19.094 | INFO     | hyvideo.inference:from_pretrained:155 - Got text-to-video model root path: ckpts
2025-04-03 09:37:19.096 | INFO     | hyvideo.inference:from_pretrained:190 - Building model...


detect you are not use the latest yunchang. Please install yunchang>=0.4.0
{'apply_final_norm': False,
 'batch_size': 1,
 'cfg_scale': 1.0,
 'decay_eta': False,
 'denoise_type': 'flow',
 'disable_autocast': False,
 'dit_weight': '/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt',
 'embedded_cfg_scale': 3.5,
 'eta_decay_power': 1.0,
 'eta_reverse': 0.9,
 'feature_path': 'feature',
 'flow_reverse': True,
 'flow_shift': 7.0,
 'flow_solver': 'euler',
 'gamma': 0.5,
 'hidden_state_skip_layer': 2,
 'infer_steps': 28,
 'inject': 0,
 'inverse_video_path': '/data/chx/BalanceCC-rename/Result/Animal/blackswan.mp4',
 'latent_channels': 16,
 'linear_schedule_end': 25,
 'load_key': 'module',
 'model': 'HYVideo-T/2-cfgdistill',
 'model_base': 'ckpts',
 'model_resolution': '540p',
 'name_suffix': '',
 'neg_prompt': None,
 'num_videos': 1,
 'precision': 'bf16',
 'prompt': 'A black swan swimming in a pond with lush greenery in the '
           'backgr

/home/chx/mySrc/HunyuanVideo_rf_inv/hyvideo/modules/fp8_optimization.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fp8_map = torch.load(fp8_map_path, map_location=lam

loading vae


/home/chx/mySrc/HunyuanVideo_rf_inv/hyvideo/vae/__init__.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(vae_ckpt, map_location=vae.device)
2025-04-03

...

 89%|████████▉ | 25/28 [01:24<00:10,  3.40s/it]

Step 25: t_curr = 456.5217590332031, t_prev = 350.0


 93%|█████████▎| 26/28 [01:28<00:06,  3.40s/it]

Step 26: t_curr = 350.0, t_prev = 205.88235473632812


 96%|█████████▋| 27/28 [01:31<00:03,  3.40s/it]
2025-04-03 09:43:08.419 | INFO     | hyvideo.inference:predict:721 - Success, time: 227.2875998020172
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-04-03 09:43:09.859 | INFO     | __main__:main:95 - Sample save to: ./results/rf_inversion/2025-04-03-09:43:08_seed42_A black swan swimming in a pond with lush greenery in the background..mp4


In [1]:
%run sample_video.py \
    --dit-weight "/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt" \
    --video-size 512 512 \
    --video-length 25 \
    --infer-steps 30 \
    --prompt "A cat walks on the grass, realistic style." \
    --seed 42 \
    --embedded-cfg-scale 6.0 \
    --flow-shift 7.0 \
    --flow-reverse \
    --use-cpu-offload \
    --use-fp8 \
    --save-path ./results

/data/chx/anaconda3/envs/HunyuanVideo-std/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-14 20:33:19.204 | INFO     | hyvideo.inference:from_pretrained:154 - Got text-to-video model root path: ckpts
2025-01-14 20:33:19.205 | INFO     | hyvideo.inference:from_pretrained:189 - Building model...


detect you are not use the latest yunchang. Please install yunchang>=0.4.0
Namespace(model='HYVideo-T/2-cfgdistill', latent_channels=16, precision='bf16', rope_theta=256, vae='884-16c-hy', vae_precision='fp16', vae_tiling=True, text_encoder='llm', text_encoder_precision='fp16', text_states_dim=4096, text_len=256, tokenizer='llm', prompt_template='dit-llm-encode', prompt_template_video='dit-llm-encode-video', hidden_state_skip_layer=2, apply_final_norm=False, text_encoder_2='clipL', text_encoder_precision_2='fp16', text_states_dim_2=768, tokenizer_2='clipL', text_len_2=77, denoise_type='flow', flow_shift=7.0, flow_reverse=True, flow_solver='euler', use_linear_quadratic_schedule=False, linear_schedule_end=25, model_base='ckpts', dit_weight='/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt', model_resolution='540p', load_key='module', use_cpu_offload=True, batch_size=1, infer_steps=30, disable_autocast=False, save_path='./results', save

/home/chx/mySrc/HunyuanVideo/hyvideo/modules/fp8_optimization.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fp8_map = torch.load(fp8_map_path, map_location=lambda sto

loading vae


/home/chx/mySrc/HunyuanVideo/hyvideo/vae/__init__.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(vae_ckpt, map_location=vae.device)
2025-01-14 20:36:

Enable sequential CPU offload.


2025-01-14 20:36:15.189 | DEBUG    | hyvideo.inference:predict:650 - 
                        height: 512
                         width: 512
                  video_length: 25
                        prompt: ['A cat walks on the grass, realistic style.']
                    neg_prompt: ['Aerial view, aerial view, overexposed, low quality, deformation, a poor composition, bad hands, bad teeth, bad eyes, bad limbs, distortion']
                          seed: 42
                   infer_steps: 30
         num_videos_per_prompt: 1
                guidance_scale: 1.0
                      n_tokens: 7168
                    flow_shift: 7.0
       embedded_guidance_scale: 6.0


self._execution_device cuda:0
cuda:0
cuda:0
Step 0: t = 1000.0
Step 1: t = 995.0980834960938
Step 2: t = 989.8989868164062
Step 3: t = 984.375
Step 4: t = 978.49462890625
Step 5: t = 972.22216796875
Step 6: t = 965.5172119140625
Step 7: t = 958.3333740234375
Step 8: t = 950.6172485351562
Step 9: t = 942.3078002929688
Step 10: t = 933.333251953125
Step 11: t = 923.611083984375
Step 12: t = 913.04345703125
Step 13: t = 901.51513671875
Step 14: t = 888.888916015625
Step 15: t = 875.0
Step 16: t = 859.6491088867188
Step 17: t = 842.5925903320312
Step 18: t = 823.5294189453125
Step 19: t = 802.0833129882812
Step 20: t = 777.77783203125
Step 21: t = 750.0
Step 22: t = 717.94873046875
Step 23: t = 680.5556030273438
Step 24: t = 636.3637084960938
Step 25: t = 583.3333740234375
Step 26: t = 518.5184936523438
Step 27: t = 437.5000305175781
Step 28: t = 333.3333435058594
Step 29: t = 194.44444274902344


100%|██████████| 30/30 [00:49<00:00,  1.64s/it]
2025-01-14 20:37:23.031 | INFO     | hyvideo.inference:predict:684 - Success, time: 67.84050464630127
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-01-14 20:37:24.919 | INFO     | __main__:main:57 - Sample save to: ./results/2025-01-14-20:37:23_seed42_A cat walks on the grass, realistic style..mp4
